<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">NBA API Data (2022-2024)</p>
</div>

<a name="Feature-Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups and Targets](#1.-Create-Team-Matchups-and-Targets)**

- [1.1. Clean Game Data](#1.1.-Clean-Game-Data)

- [1.2. Reshape to Game Matchups](#1.2.-Reshape-to-Game-Matchups)

- [1.3. Create Target Variables](#1.3.-Create-Target-Variables)

**[2. Create Rolling Window Statistics](#2.-Create-Rolling-Window-Statistics)**

# Setup

[Return to top](#Feature-Engineering)

In [2]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

# Data

[Return to top](#Feature-Engineering)

In [5]:
scoring_box_stats_df = pd.read_csv('../../data/original/nba_scoring_statistics_2021_2024.csv')
scoring_box_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
0,22101221,1610612745,Rockets,HOU,Houston,240.000000:00,0.483,0.517,0.421,0.018,0.447,0.114,0.132,0.132,0.404,0.500,0.500,0.706,0.294,0.585,0.415
1,22101221,1610612737,Hawks,ATL,Atlanta,240.000000:00,0.563,0.437,0.369,0.062,0.485,0.108,0.146,0.123,0.308,0.500,0.500,0.810,0.190,0.644,0.356
2,22101207,1610612748,Heat,MIA,Miami,240.000000:00,0.639,0.361,0.513,0.088,0.292,0.133,0.195,0.142,0.425,0.448,0.552,0.636,0.364,0.500,0.500
3,22101207,1610612737,Hawks,ATL,Atlanta,240.000000:00,0.614,0.386,0.624,0.239,0.220,0.110,0.156,0.202,0.385,0.324,0.676,0.750,0.250,0.405,0.595
4,22101192,1610612764,Wizards,WAS,Washington,240.000000:00,0.593,0.407,0.621,0.194,0.291,0.126,0.087,0.078,0.427,0.500,0.500,1.000,0.000,0.619,0.381


In [6]:
box_score_df = pd.read_csv('../../data/original/nba_games_box_scores_2022_2024.csv')

In [7]:
box_score_df.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7520,22023,1610612764,WAS,Washington Wizards,22300642,2024-01-27,WAS @ DET,W,240,118,45,100,0.450,11,34.0,0.324,17,21,0.810,16.0,34.0,50.0,26,10.0,4,9,19,14.0
7521,22023,1610612764,WAS,Washington Wizards,22300665,2024-01-29,WAS @ SAS,W,240,118,46,86,0.535,9,25.0,0.360,17,24,0.708,14.0,31.0,45.0,32,9.0,8,18,15,5.0
7522,22023,1610612764,WAS,Washington Wizards,22300676,2024-01-31,WAS vs. LAC,L,239,109,45,97,0.464,9,29.0,0.310,10,15,0.667,12.0,33.0,45.0,19,4.0,10,13,19,-16.0
7523,22023,1610612764,WAS,Washington Wizards,22300689,2024-02-02,WAS vs. MIA,L,239,102,37,90,0.411,11,42.0,0.262,17,21,0.810,6.0,37.0,43.0,28,5.0,4,8,25,-8.0
7524,22023,1610612764,WAS,Washington Wizards,22300705,2024-02-04,WAS vs. PHX,L,240,112,47,96,0.490,7,32.0,0.219,11,17,0.647,13.0,22.0,35.0,32,11.0,4,18,19,-28.0


In [9]:
#formats the minutes field
scoring_box_stats_df['MIN'] = scoring_box_stats_df['MIN'].str.slice(0, 3)
scoring_box_stats_df['MIN'] = scoring_box_stats_df['MIN'].astype(int)
#creates team name field to match games_df
scoring_box_stats_df['TEAM_NAME'] = scoring_box_stats_df['TEAM_CITY'] + " " + scoring_box_stats_df['TEAM_NAME']
scoring_box_stats_df.drop(['TEAM_CITY'], inplace=True, axis=1)
scoring_box_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
0,22101221,1610612745,Houston Rockets,HOU,240,0.483,0.517,0.421,0.018,0.447,0.114,0.132,0.132,0.404,0.500,0.500,0.706,0.294,0.585,0.415
1,22101221,1610612737,Atlanta Hawks,ATL,240,0.563,0.437,0.369,0.062,0.485,0.108,0.146,0.123,0.308,0.500,0.500,0.810,0.190,0.644,0.356
2,22101207,1610612748,Miami Heat,MIA,240,0.639,0.361,0.513,0.088,0.292,0.133,0.195,0.142,0.425,0.448,0.552,0.636,0.364,0.500,0.500
3,22101207,1610612737,Atlanta Hawks,ATL,240,0.614,0.386,0.624,0.239,0.220,0.110,0.156,0.202,0.385,0.324,0.676,0.750,0.250,0.405,0.595
4,22101192,1610612764,Washington Wizards,WAS,240,0.593,0.407,0.621,0.194,0.291,0.126,0.087,0.078,0.427,0.500,0.500,1.000,0.000,0.619,0.381


In [10]:
scoring_box_stats_df = pd.merge(scoring_box_stats_df, box_score_df[['SEASON_ID','GAME_DATE','MATCHUP', 'GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

scoring_box_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,SEASON_ID,GAME_DATE,MATCHUP
0,22101221,1610612745,Houston Rockets,HOU,240,0.483,0.517,0.421,0.018,0.447,0.114,0.132,0.132,0.404,0.500,0.500,0.706,0.294,0.585,0.415,22021,2022-04-10,HOU vs. ATL
1,22101221,1610612737,Atlanta Hawks,ATL,240,0.563,0.437,0.369,0.062,0.485,0.108,0.146,0.123,0.308,0.500,0.500,0.810,0.190,0.644,0.356,22021,2022-04-10,ATL @ HOU
2,22101207,1610612748,Miami Heat,MIA,240,0.639,0.361,0.513,0.088,0.292,0.133,0.195,0.142,0.425,0.448,0.552,0.636,0.364,0.500,0.500,22021,2022-04-08,MIA vs. ATL
3,22101207,1610612737,Atlanta Hawks,ATL,240,0.614,0.386,0.624,0.239,0.220,0.110,0.156,0.202,0.385,0.324,0.676,0.750,0.250,0.405,0.595,22021,2022-04-08,ATL @ MIA
4,22101192,1610612764,Washington Wizards,WAS,240,0.593,0.407,0.621,0.194,0.291,0.126,0.087,0.078,0.427,0.500,0.500,1.000,0.000,0.619,0.381,22021,2022-04-06,WAS @ ATL


<a name="1.-Create-Team-Matchups-and-Targets"></a>
# 1. Create Team Matchups and Targets

[Return to top](#Feature-Engineering)

<a name="1.1.-Clean-Game-Data"></a>
## 1.1. Clean Game Data

[Return to top](#Feature-Engineering)

We need to do three key things to clean the data:

1. Remove games with team aggregated game times of less than 238 minutes (which will remove exhibition matches).
2. Retain only games that are part of the regular season.
3. Remove any orphans (i.e., game IDs that do not have a partner) when reshaping to matchups.

Last 3 NBA regular seasons start and end dates:

- 2021-22 season: 2021-10-19 to 2022-04-10
- 2022-23 season: 2022-10-18 to 2023-04-09
- 2023-24 season: 2023-10-24 to 2024-04-14

In [11]:
# last 3 seasons start and end dates and labels
season_start_dates = ['2021-10-19', '2022-10-18', '2023-10-24']
season_end_dates   = ['2022-04-10', '2023-04-09', '2024-04-14']
season_labels      = ['2021-22', '2022-23', '2023-24']

In [12]:
# clean up the data
scoring_box_stats_df_cleaned = utl.clean_team_bs_data(scoring_box_stats_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 739 games


In [13]:
# clean up the data
box_score_df_cleaned = utl.clean_team_bs_data(box_score_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 736 games


In [14]:
#get GAME_DATE, MATCHUP, GAME_ID, TEAM_ABBREVIATION fields from games_df
scoring_box_stats_df_cleaned = pd.merge(scoring_box_stats_df_cleaned, box_score_df_cleaned[['WL','PTS','GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION', 'PLUS_MINUS']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

scoring_box_stats_df_cleaned.sort_values(by=['PTS'])

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,MIN,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,SEASON_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS
890,22100075,1610612742,Dallas Mavericks,DAL,240,0.526,0.474,0.400,0.107,0.320,0.107,0.280,0.267,0.293,0.533,0.467,0.750,0.250,0.609,0.391,2021-22,2021-10-29,DAL @ DEN,L,75,-31.0
242,22100595,1610612752,New York Knicks,NYK,240,0.608,0.392,0.480,0.160,0.440,0.040,0.080,0.173,0.320,0.222,0.778,0.818,0.182,0.448,0.552,2021-22,2022-01-08,NYK @ BOS,L,75,-24.0
224,22100717,1610612758,Sacramento Kings,SAC,240,0.653,0.347,0.613,0.160,0.240,0.187,0.147,0.147,0.453,0.435,0.565,0.833,0.167,0.517,0.483,2021-22,2022-01-25,SAC @ BOS,L,75,-53.0
6222,22300529,1610612757,Portland Trail Blazers,POR,240,0.515,0.485,0.442,0.052,0.429,0.143,0.130,0.156,0.390,0.412,0.588,0.818,0.182,0.571,0.429,2023-24,2024-01-11,POR @ OKC,L,77,-62.0
727,22100257,1610612741,Chicago Bulls,CHI,240,0.671,0.329,0.649,0.234,0.234,0.091,0.117,0.065,0.416,0.320,0.680,0.833,0.167,0.419,0.581,2021-22,2021-11-22,CHI vs. IND,L,77,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,22201230,1610612744,Golden State Warriors,GSW,240,0.490,0.510,0.395,0.013,0.516,0.166,0.089,0.159,0.382,0.774,0.226,0.852,0.148,0.810,0.190,2022-23,2023-04-09,GSW @ POR,W,157,56.0
4975,22300039,1610612754,Indiana Pacers,IND,240,0.521,0.479,0.446,0.038,0.420,0.127,0.134,0.121,0.408,0.600,0.400,0.636,0.364,0.614,0.386,2023-24,2023-11-21,IND @ ATL,W,157,5.0
2049,22100723,1610612766,Charlotte Hornets,CHA,240,0.516,0.484,0.380,0.038,0.456,0.190,0.165,0.190,0.342,0.567,0.433,0.833,0.167,0.685,0.315,2021-22,2022-01-26,CHA @ IND,W,158,32.0
3751,22200902,1610612746,LA Clippers,LAC,290,0.541,0.459,0.377,0.103,0.446,0.097,0.177,0.154,0.274,0.455,0.545,0.923,0.077,0.661,0.339,2022-23,2023-02-24,LAC vs. SAC,L,175,-1.0


<a name="1.2.-Reshape-to-Game-Matchups"></a>
## 1.2. Reshape to Game Matchups

[Return to top](#Feature-Engineering)

In [15]:
# identify non-stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP']

# reshape team box score data to wide format so each row is a game matchup
scoring_box_stats_matchups_df = utl.reshape_team_bs_to_matchups(scoring_box_stats_df_cleaned, non_stats_cols)

Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 728 games


In [16]:
scoring_box_stats_matchups_df.head()

,GAME_ID,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_PCT_FGA_2PT,HOME_PCT_FGA_3PT,HOME_PCT_PTS_2PT,HOME_PCT_PTS_2PT_MR,HOME_PCT_PTS_3PT,HOME_PCT_PTS_FB,HOME_PCT_PTS_FT,HOME_PCT_PTS_OFF_TOV,HOME_PCT_PTS_PAINT,HOME_PCT_AST_2PM,HOME_PCT_UAST_2PM,HOME_PCT_AST_3PM,HOME_PCT_UAST_3PM,HOME_PCT_AST_FGM,HOME_PCT_UAST_FGM,SEASON_ID,GAME_DATE,HOME_WL,HOME_PTS,HOME_PLUS_MINUS,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_PCT_FGA_2PT,AWAY_PCT_FGA_3PT,AWAY_PCT_PTS_2PT,AWAY_PCT_PTS_2PT_MR,AWAY_PCT_PTS_3PT,AWAY_PCT_PTS_FB,AWAY_PCT_PTS_FT,AWAY_PCT_PTS_OFF_TOV,AWAY_PCT_PTS_PAINT,AWAY_PCT_AST_2PM,AWAY_PCT_UAST_2PM,AWAY_PCT_AST_3PM,AWAY_PCT_UAST_3PM,AWAY_PCT_AST_FGM,AWAY_PCT_UAST_FGM,AWAY_WL,AWAY_PTS,AWAY_PLUS_MINUS
0,22101221,1610612745,Houston Rockets,HOU,240,0.483,0.517,0.421,0.018,0.447,0.114,0.132,0.132,0.404,0.500,0.500,0.706,0.294,0.585,0.415,2021-22,2022-04-10,L,114,-16.0,1610612737,Atlanta Hawks,ATL,240,0.563,0.437,0.369,0.062,0.485,0.108,0.146,0.123,0.308,0.500,0.500,0.810,0.190,0.644,0.356,W,130,16.0
1,22101207,1610612748,Miami Heat,MIA,240,0.639,0.361,0.513,0.088,0.292,0.133,0.195,0.142,0.425,0.448,0.552,0.636,0.364,0.500,0.500,2021-22,2022-04-08,W,113,4.0,1610612737,Atlanta Hawks,ATL,240,0.614,0.386,0.624,0.239,0.220,0.110,0.156,0.202,0.385,0.324,0.676,0.750,0.250,0.405,0.595,L,109,-4.0
2,22101192,1610612737,Atlanta Hawks,ATL,240,0.576,0.424,0.458,0.119,0.407,0.093,0.136,0.136,0.339,0.556,0.444,0.688,0.313,0.605,0.395,2021-22,2022-04-06,W,118,15.0,1610612764,Washington Wizards,WAS,240,0.593,0.407,0.621,0.194,0.291,0.126,0.087,0.078,0.427,0.500,0.500,1.000,0.000,0.619,0.381,L,103,-15.0
3,22101182,1610612761,Toronto Raptors,TOR,240,0.653,0.347,0.593,0.068,0.229,0.093,0.178,0.076,0.525,0.371,0.629,0.778,0.222,0.455,0.545,2021-22,2022-04-05,W,118,10.0,1610612737,Atlanta Hawks,ATL,240,0.606,0.394,0.537,0.148,0.389,0.102,0.074,0.083,0.389,0.621,0.379,0.929,0.071,0.721,0.279,L,108,-10.0
4,22101163,1610612737,Atlanta Hawks,ATL,240,0.523,0.477,0.377,0.082,0.320,0.098,0.303,0.123,0.295,0.348,0.652,0.769,0.231,0.500,0.500,2021-22,2022-04-02,W,122,7.0,1610612751,Brooklyn Nets,BKN,240,0.626,0.374,0.435,0.122,0.443,0.130,0.122,0.104,0.313,0.400,0.600,0.412,0.588,0.405,0.595,L,115,-7.0


<a name="1.3.-Create-Target-Variables"></a>
## 1.3. Create Target Variables

[Return to top](#Feature-Engineering)

There are three targets of interest:

1. **Total Game Points (over / under):** This can be calculated as the sum of `HOME_PTS + AWAY_PTS`.
2. **Difference in Game Points (plus / minus):** This can be calculated in relation to the home team as the following difference: `HOME_PTS - AWAY_PTS`.
3. **Game Winner (moneyline):** This can be defined in relation to the home team using the `HOME_WL` column, where a win for the home team is equal to 1 and a loss for the home team equal to 0. We will create a new column called `GAME_RESULT` for this indicator.

In [17]:
# create the above three target variables
scoring_box_stats_matchups_df = utl.create_target_variables(scoring_box_stats_matchups_df, 'HOME_WL', 'HOME_PTS', 'AWAY_PTS')

In [18]:
scoring_box_stats_matchups_df[['GAME_DATE', 'GAME_ID',  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_PTS', 'AWAY_PTS', 'GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS']].tail()

,GAME_DATE,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,HOME_PTS,AWAY_PTS,GAME_RESULT,TOTAL_PTS,PLUS_MINUS
3166,2023-11-22,22300225,Charlotte Hornets,Washington Wizards,117,114,1,231,3.0
3167,2023-11-10,22300009,Washington Wizards,Charlotte Hornets,117,124,0,241,-7.0
3168,2023-11-08,22300157,Charlotte Hornets,Washington Wizards,116,132,0,248,-16.0
3169,2024-01-24,22300619,Detroit Pistons,Charlotte Hornets,113,106,1,219,7.0
3170,2023-10-27,22300077,Charlotte Hornets,Detroit Pistons,99,111,0,210,-12.0


<a name="2.-Create-Rolling-Window-Statistics"></a>
# 2. Create Rolling Window Statistics

[Return to top](#Feature-Engineering)

Here we create average box scores for each team over a rolling window of the previous $n$-games.

In [20]:
# identify stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID',
                  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_WL', 'AWAY_WL', 'HOME_MIN', 
                  'AWAY_MIN', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']
stats_cols = [col for col in scoring_box_stats_matchups_df.columns if col not in non_stats_cols]

In [27]:
# calculate rolling averages for each statistic and add them to the DataFrame
scoring_box_stats_matchups_roll_df = utl.process_rolling_stats(
    scoring_box_stats_matchups_df, 
    stats_cols, 
    target_cols=['GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS'],
    window_size=5,   # the number of games to include in the rolling window
    min_obs=1,       # the minimum number of observations present within the window to yield an aggregate value
    stratify_by_season=True,  # should the rolling calculations be reset at the start of each new season or be contiguous across seasons? 
    exclude_initial_games=0   # number of initial games to exclude from the rolling averages (optionally by season)
)

In [28]:
scoring_box_stats_matchups_roll_df.tail()

,GAME_ID,GAME_RESULT,TOTAL_PTS,PLUS_MINUS,HOME_TEAM_NAME,SEASON_ID,GAME_DATE,ROLL_HOME_PCT_FGA_2PT,ROLL_HOME_PCT_FGA_3PT,ROLL_HOME_PCT_PTS_2PT,ROLL_HOME_PCT_PTS_2PT_MR,ROLL_HOME_PCT_PTS_3PT,ROLL_HOME_PCT_PTS_FB,ROLL_HOME_PCT_PTS_FT,ROLL_HOME_PCT_PTS_OFF_TOV,ROLL_HOME_PCT_PTS_PAINT,ROLL_HOME_PCT_AST_2PM,ROLL_HOME_PCT_UAST_2PM,ROLL_HOME_PCT_AST_3PM,ROLL_HOME_PCT_UAST_3PM,ROLL_HOME_PCT_AST_FGM,ROLL_HOME_PCT_UAST_FGM,ROLL_HOME_PTS,AWAY_TEAM_NAME,ROLL_AWAY_PCT_FGA_2PT,ROLL_AWAY_PCT_FGA_3PT,ROLL_AWAY_PCT_PTS_2PT,ROLL_AWAY_PCT_PTS_2PT_MR,ROLL_AWAY_PCT_PTS_3PT,ROLL_AWAY_PCT_PTS_FB,ROLL_AWAY_PCT_PTS_FT,ROLL_AWAY_PCT_PTS_OFF_TOV,ROLL_AWAY_PCT_PTS_PAINT,ROLL_AWAY_PCT_AST_2PM,ROLL_AWAY_PCT_UAST_2PM,ROLL_AWAY_PCT_AST_3PM,ROLL_AWAY_PCT_UAST_3PM,ROLL_AWAY_PCT_AST_FGM,ROLL_AWAY_PCT_UAST_FGM,ROLL_AWAY_PTS
2540,22300703,0,218,-16.0,San Antonio Spurs,2023-24,2024-02-03,0.631,0.369,0.575,0.087,0.293,0.178,0.133,0.133,0.487,0.644,0.356,0.962,0.038,0.725,0.275,110.6,Cleveland Cavaliers,0.561,0.439,0.472,0.072,0.398,0.143,0.130,0.137,0.400,0.493,0.507,0.853,0.147,0.623,0.377,115.6
3077,22300705,0,252,-28.0,Washington Wizards,2023-24,2024-02-04,0.657,0.343,0.588,0.053,0.255,0.145,0.157,0.131,0.535,0.666,0.334,0.895,0.105,0.716,0.284,106.0,Phoenix Suns,0.654,0.346,0.588,0.158,0.287,0.131,0.124,0.107,0.431,0.530,0.470,0.789,0.211,0.592,0.408,120.6
3017,22300704,0,210,-12.0,Detroit Pistons,2023-24,2024-02-04,0.624,0.376,0.523,0.086,0.321,0.088,0.156,0.109,0.436,0.534,0.466,0.764,0.236,0.604,0.396,115.0,Orlando Magic,0.645,0.355,0.542,0.068,0.281,0.142,0.177,0.190,0.474,0.497,0.503,0.838,0.162,0.587,0.413,111.0
3036,22300707,0,214,-18.6,Charlotte Hornets,2023-24,2024-02-04,0.594,0.406,0.528,0.064,0.336,0.146,0.136,0.144,0.464,0.558,0.442,0.844,0.156,0.644,0.356,103.4,Indiana Pacers,0.613,0.387,0.511,0.086,0.375,0.105,0.113,0.119,0.425,0.587,0.413,0.888,0.112,0.678,0.322,116.0
2492,22300706,1,222,40.0,Boston Celtics,2023-24,2024-02-04,0.527,0.473,0.454,0.087,0.433,0.126,0.112,0.109,0.367,0.473,0.527,0.788,0.212,0.594,0.406,113.8,Memphis Grizzlies,0.573,0.427,0.462,0.034,0.392,0.140,0.146,0.139,0.427,0.575,0.425,0.926,0.074,0.696,0.304,104.4


In [29]:
# write out the matchups with rolling features
scoring_box_stats_matchups_roll_df.to_csv('../../data/processed/nba_team_matchups_rolling_scoring_box_stats_2021_2024_r05.csv', index=False)